# 평가
1. 인사정보 평가
    - 임베딩 모델 : KURE-V1
    - 리랭크 모델 : BGE-RERANKER-KO
    - 질문지 : GPT-4O 생성 102개질문지
    - LLM : GPT-4O-mini
    - topk1~5 측정

2. 평가 결과
    - topk 1~5 F1/EM score : 41.11/34.04 46.52/37.77 48.09/38.30 46.62/38.30 48.59/39.36
    


In [4]:
# 라이브러리 및 환경설정

from openai import OpenAI
import json
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import evaluate

client = OpenAI(api_key="key")  

# 임베딩모델 로드 ( 허깅페이스 )
embedding = HuggingFaceEmbeddings(
    model_name="work1/models/kure_v1",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)


C:\Users\kdy\AppData\Local\Temp\ipykernel_29600\794522863.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(


In [7]:
# 내부 규정 평가 - 리랭커 top-k 실험

from rag_eval import run_custom_prompt_eval_group

my_custom_prompt = (
    """질문에 정확히 대응하는 정답만 말하세요. 예시:
Q: 퇴직금은 어떻게 계산되나요? → 평균임금
Q: 지급 대상은 누구인가요? → 퇴직자
이와 같은 방식으로 단답형 정답만 출력하세요. 정답이 없으면 '문서에 없습니다'라고 하세요."""
)

# ✅ Top-k 실험 루프
for k in range(1, 6):  # top_k=1~5
    print(f"\n🔍 [실험] Top-{k} 문서 → 리랭커 → GPT 응답 평가")
    run_custom_prompt_eval_group(
        client=client,
        embedding=embedding,
        eval_path="eval_questions_gpt.jsonl",
        top_k=k,
        system_prompt=my_custom_prompt
    )



🔍 [실험] Top-1 문서 → 리랭커 → GPT 응답 평가


100%|██████████| 188/188 [03:03<00:00,  1.02it/s]


📊 평균 F1: 41.11
📊 평균 EM: 34.04

🔍 [실험] Top-2 문서 → 리랭커 → GPT 응답 평가


100%|██████████| 188/188 [02:34<00:00,  1.22it/s]


📊 평균 F1: 46.52
📊 평균 EM: 37.77

🔍 [실험] Top-3 문서 → 리랭커 → GPT 응답 평가


...

📊 평균 F1: 48.09
📊 평균 EM: 38.30

🔍 [실험] Top-4 문서 → 리랭커 → GPT 응답 평가


100%|██████████| 188/188 [06:20<00:00,  2.03s/it]


📊 평균 F1: 46.62
📊 평균 EM: 38.30

🔍 [실험] Top-5 문서 → 리랭커 → GPT 응답 평가


100%|██████████| 188/188 [08:08<00:00,  2.60s/it]

📊 평균 F1: 48.59
📊 평균 EM: 39.36
